In [2]:
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import pandas as pd
from sklearn.metrics import classification_report
import joblib
from tdc import Evaluator

In [3]:
#Get data
train_data = pd.read_csv('../../data/MorganCount/tox21_train_featurized.csv').dropna()
valid_data = pd.read_csv('../../data/MorganCount/tox21_valid_featurized.csv').dropna()
test_data = pd.read_csv('../../data/MorganCount/tox21_test_featurized.csv').dropna()

#get splits
X_train, y_train = train_data.filter(regex='^dim_.*'), train_data['Y']
X_test, y_test = test_data.filter(regex='^dim_.*'), test_data['Y']
X_valid, y_valid = valid_data.filter(regex='^dim_.*'), valid_data['Y']

#Use smote to oversample minority class
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

In [ ]:
#Train the model
model = XGBClassifier(
    n_estimators=200,
    max_depth=7,               # Balance complexity
    learning_rate=0.05,        # Slower learning
    scale_pos_weight=8,        # Adjust for class imbalance 
    random_state=42,
    early_stopping_rounds=10,   #prevent overfitting
)

model.fit(X_res, y_res , eval_set=[(X_valid, y_valid)], verbose=True)

In [6]:
#get prediction result
y_test_pred = model.predict(X_test)
y_val_pred = model.predict(X_valid)

# Display metrics
print(classification_report(y_test, y_test_pred))
print(classification_report(y_valid, y_val_pred))

              precision    recall  f1-score   support

         0.0       0.94      0.87      0.90       979
         1.0       0.49      0.69      0.57       181

    accuracy                           0.84      1160
   macro avg       0.71      0.78      0.74      1160
weighted avg       0.87      0.84      0.85      1160

              precision    recall  f1-score   support

         0.0       0.94      0.85      0.90       472
         1.0       0.55      0.78      0.65       109

    accuracy                           0.84       581
   macro avg       0.75      0.82      0.77       581
weighted avg       0.87      0.84      0.85       581



In [46]:
from typing import Dict, Any
def evaluate_model(y_true, y_pred_proba, threshold: float = 0.5) -> Dict[str, float]:
    metrics = {
        'ROC-AUC': {'name': 'ROC-AUC', 'kwargs': {}},
        'PR-AUC': {'name': 'PR-AUC', 'kwargs': {}},
        'Accuracy': {'name': 'Accuracy', 'kwargs': {'threshold': threshold}},
        'Precision': {'name': 'Precision', 'kwargs': {'threshold': threshold}},
        'Recall': {'name': 'Recall', 'kwargs': {'threshold': threshold}},
        'F1': {'name': 'F1', 'kwargs': {'threshold': threshold}}
    }
    
    results = {}
    for metric_name, config in metrics.items():
        evaluator = Evaluator(name=config['name'])
        score = evaluator(y_true, y_pred_proba, **config['kwargs'])
        results[metric_name] = score
        print(f"{metric_name}: {score:.4f}")
    
    return results

y_pred_proba = model.predict_proba(X_test)[:, 1]
y_true = y_test

evaluation_results = evaluate_model(y_true, y_pred_proba)

ROC-AUC: 0.8859
PR-AUC: 0.6369
Accuracy: 0.8509
Precision: 0.5161
Recall: 0.7072
F1: 0.5967


In [47]:
#Save model

model = model
model_filename = 'Morgan_trained_model.joblib'
joblib.dump(model, model_filename)

['Morgan_trained_model.joblib']